**Introduction**

This project uses the “New York Stock Exchange” dataset published on Kaggle (files: prices.csv, prices-split-adjusted.csv, securities.csv, fundamentals.csv). It contains daily OHLCV prices (2010–2016) for S&P 500 constituents, plus sector classifications and company fundamentals (annual financial statement metrics). It’s interesting because it lets us connect market behavior (returns, volatility, liquidity) with business fundamentals (growth, profitability, leverage) using SQL only.

**How to access / download**
- Go to the Kaggle page and download the dataset files (CSV) through this link : https://www.kaggle.com/datasets/dgawlik/nyse?resource=download.
- Import into your SQL engine (SQLite / Postgres / MySQL). Example (SQLite): create tables then .mode csv + .import.

**Note on finance definitions used below**
- Returns are computed from split-adjusted close (from prices-split-adjusted.csv) and use first vs last trading day of 2016. This excludes dividends (so it’s not total shareholder return).
- “Sharpe-like” uses 0% risk-free and daily mean/vol scaled to annual.

**1) Liquidity: Which stocks dominated trading in 2016 (avg daily $ volume)?**

_Question :_ Identify the most liquid names (useful for market impact, execution cost, and “tradability”).

In [2]:
-- Top 10 most liquid stocks in 2016 by average daily $ volume (Close * Volume)
WITH daily_2016 AS (
    SELECT
        symbol,
        date AS trade_date,
        close,
        volume,
        close * volume AS dollar_volume
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = 2016
),
liquidity AS (
    SELECT
        symbol,
        AVG(volume) AS avg_daily_volume,
        AVG(dollar_volume) AS avg_daily_dollar_volume,
        COUNT(*) AS trading_days_2016
    FROM daily_2016
    GROUP BY symbol
    HAVING trading_days_2016 >= 200
)
SELECT
    l.symbol,
    s.security,
    s."GICS Sector" as gics_sector,
    ROUND(l.avg_daily_volume / 1e6, 2) AS avg_daily_volume_m_shares,
    ROUND(l.avg_daily_dollar_volume / 1e9, 2) AS avg_daily_dollar_volume_usd_bn
FROM liquidity l
LEFT JOIN 'securities.csv' s
    ON s."Ticker symbol" = l.symbol
ORDER BY l.avg_daily_dollar_volume DESC
LIMIT 10;


,symbol,Security,gics_sector,avg_daily_volume_m_shares,avg_daily_dollar_volume_usd_bn
0,AAPL,Apple Inc.,Information Technology,38.42,3.98
1,FB,Facebook,Information Technology,25.45,2.92
2,AMZN,Amazon.com Inc,Consumer Discretionary,4.12,2.81
3,MSFT,Microsoft Corp.,Information Technology,31.02,1.70
4,BAC,Bank of America Corp,Financials,108.27,1.67
5,GOOGL,Alphabet Inc Class A,Information Technology,1.97,1.49
6,GOOG,Alphabet Inc Class C,Information Technology,1.83,1.35
7,NFLX,Netflix Inc.,Information Technology,12.81,1.28
8,WFC,Wells Fargo,Financials,22.95,1.12
9,JPM,JPMorgan Chase & Co.,Financials,17.06,1.11


**_Insight :_**

- Liquidity is concentrated in mega-caps (AAPL/FB/AMZN/MSFT) and large banks (BAC/WFC/JPM).
- Sector-wise, Information Technology + Financials dominate the liquidity leaderboard. These sectors are typically easiest to trade with lower spreads/impact.

**2) Performance: Which sectors had the best average total return in 2016?**

_Question :_ Compare sector performance using average stock total returns (2016).

In [3]:
-- Sector performance in 2016 (average stock total return, min/max within sector)
WITH prices_2016 AS (
    SELECT
        symbol,
        date AS trade_date,
        close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = '2016'
),
first_last AS (
    SELECT
        symbol,
        MIN(trade_date) AS first_date,
        MAX(trade_date) AS last_date
    FROM prices_2016
    GROUP BY symbol
),
returns AS (
    SELECT
        fl.symbol,
        (p_last.close / p_first.close - 1.0) AS total_return_2016
    FROM first_last fl
    INNER JOIN prices_2016 p_first
        ON p_first.symbol = fl.symbol AND p_first.trade_date = fl.first_date
    INNER JOIN prices_2016 p_last
        ON p_last.symbol = fl.symbol AND p_last.trade_date = fl.last_date
)
SELECT
    s."GICS Sector" as gics_sector,
    COUNT(*) AS n_stocks,
    ROUND(AVG(r.total_return_2016) * 100, 2) AS avg_total_return_pct_2016,
    ROUND(MIN(r.total_return_2016) * 100, 2) AS worst_stock_return_pct,
    ROUND(MAX(r.total_return_2016) * 100, 2) AS best_stock_return_pct
FROM returns r
INNER JOIN 'securities.csv' s
    ON s."Ticker symbol" = r.symbol
GROUP BY gics_sector
HAVING n_stocks >= 5
ORDER BY avg_total_return_pct_2016 DESC;


,gics_sector,n_stocks,avg_total_return_pct_2016,worst_stock_return_pct,best_stock_return_pct
0,Energy,36,31.23,-18.35,126.47
1,Materials,25,23.53,-22.27,101.37
2,Financials,62,22.14,-30.23,64.40
3,Industrials,69,21.53,-34.64,154.71
4,Information Technology,68,19.48,-51.90,229.75
5,Utilities,28,14.95,-1.68,34.87
6,Consumer Staples,36,4.20,-26.61,36.35
7,Consumer Discretionary,84,3.71,-63.53,62.18
8,Real Estate,29,2.92,-19.45,28.95
9,Telecommunications Services,5,2.75,-27.31,23.81


**_Insight :_**

- Energy leads strongly in 2016 → typical “rebound” behavior after a weak 2015 oil environment.
- Health Care has the weakest average sector return but contains big dispersion (best stock +145% vs worst -72%).

**3) Outliers: Who were the biggest winners and losers in 2016?**

_Question :_ Find extreme stock-level outliers (useful for event/risk studies and portfolio tail analysis).

In [4]:
-- Top 5 and bottom 5 stock total returns in 2016
WITH prices_2016 AS (
    SELECT symbol, date AS trade_date, close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = '2016'
),
first_last AS (
    SELECT symbol, MIN(trade_date) AS first_date, MAX(trade_date) AS last_date
    FROM prices_2016
    GROUP BY symbol
),
returns AS (
    SELECT
        fl.symbol,
        (p_last.close / p_first.close - 1.0) AS total_return_2016
    FROM first_last fl
    INNER JOIN prices_2016 p_first
        ON p_first.symbol = fl.symbol AND p_first.trade_date = fl.first_date
    INNER JOIN prices_2016 p_last
        ON p_last.symbol = fl.symbol AND p_last.trade_date = fl.last_date
),
enriched AS (
    SELECT
        r.symbol,
        s.security,
        s."GICS sector" AS gics_sector,
        ROUND(r.total_return_2016 * 100, 2) AS total_return_pct_2016
    FROM returns r
    LEFT JOIN 'securities.csv' s
        ON s."Ticker symbol" = r.symbol
),
top5 AS (
    SELECT * FROM enriched
    ORDER BY total_return_pct_2016 DESC
    LIMIT 5
),
bottom5 AS (
    SELECT * FROM enriched
    ORDER BY total_return_pct_2016 ASC
    LIMIT 5
)
SELECT 'TOP_5' AS bucket, * FROM top5
UNION ALL
SELECT 'BOTTOM_5' AS bucket, * FROM bottom5
ORDER BY bucket, total_return_pct_2016 DESC;

,bucket,symbol,Security,gics_sector,total_return_pct_2016
0,BOTTOM_5,PRGO,Perrigo,Health Care,-41.59
1,BOTTOM_5,TRIP,TripAdvisor,Consumer Discretionary,-44.12
2,BOTTOM_5,FSLR,First Solar Inc,Information Technology,-51.90
3,BOTTOM_5,UAA,Under Armour,Consumer Discretionary,-63.53
4,BOTTOM_5,ENDP,Endo International,Health Care,-72.47
5,TOP_5,NVDA,Nvidia Corporation,Information Technology,229.75
6,TOP_5,ARNC,Arconic Inc,Industrials,154.71
7,TOP_5,EVHC,Envision Healthcare Corp,Health Care,145.12
8,TOP_5,OKE,ONEOK,Energy,126.47
9,TOP_5,FCX,Freeport-McMoran Cp & Gld,Materials,101.37


**_Insight :_**

- Biggest winners are concentrated in cyclical rebound sectors (Energy/Materials) + a major tech winner (NVDA).
- Biggest losers are mainly in Consumer Discretionary and Health Care → highlights idiosyncratic/company-specific risks.

**4) Risk: Which stocks were most volatile in 2016?**

_Question :_ Identify the highest-risk names (annualized volatility).

In [5]:
-- Top 10 most volatile stocks in 2016 using daily returns
WITH daily_2016 AS (
    SELECT
        symbol,
        date AS trade_date,
        close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = '2016'
),
daily_returns AS (
    SELECT
        symbol,
        trade_date,
        (close / LAG(close) OVER (PARTITION BY symbol ORDER BY trade_date) - 1.0) AS daily_return
    FROM daily_2016
),
vol AS (
    SELECT
        symbol,
        COUNT(daily_return) AS n_returns,
        AVG(daily_return) AS avg_daily_return,
        SQRT(AVG(daily_return * daily_return) - AVG(daily_return) * AVG(daily_return)) AS daily_vol
    FROM daily_returns
    WHERE daily_return IS NOT NULL
    GROUP BY symbol
    HAVING n_returns >= 200
)
SELECT
    v.symbol,
    s.security,
    s."GICS sector" AS gics_sector,
    ROUND(v.avg_daily_return * 100, 4) AS avg_daily_return_pct,
    ROUND(v.daily_vol * SQRT(252) * 100, 2) AS annualized_vol_pct
FROM vol v
LEFT JOIN 'securities.csv' s
    ON s."Ticker symbol" = v.symbol
ORDER BY annualized_vol_pct DESC
LIMIT 10;

,symbol,Security,gics_sector,avg_daily_return_pct,annualized_vol_pct
0,EVHC,Envision Healthcare Corp,Health Care,0.7798,214.11
1,ARNC,Arconic Inc,Industrials,0.7904,210.90
2,CHK,Chesapeake Energy,Energy,0.3760,109.67
3,ENDP,Endo International,Health Care,-0.3594,83.95
4,SWN,Southwestern Energy,Energy,0.2705,82.98
5,WMB,Williams Cos.,Energy,0.2021,81.02
6,FCX,Freeport-McMoran Cp & Gld,Materials,0.4075,80.49
7,MRO,Marathon Oil Corp.,Energy,0.2103,68.09
8,RIG,Transocean,Energy,0.1507,66.68
9,MUR,Murphy Oil,Energy,0.2059,65.43


**_Insight :_**

- Volatility is dominated by Energy names (oil sensitivity) plus a couple of idiosyncratic outliers.
- High vol can create opportunity or unacceptable drawdown risk depending on our mandate.

**5) Risk-adjusted performance: Which sectors had the best “Sharpe-like” in 2016?**

_Question :_ Compare sectors on return per unit of risk.

In [6]:
-- Sector "Sharpe-like" ratio in 2016 (mean daily return / daily vol, annualized)
WITH daily_2016 AS (
    SELECT symbol, date AS trade_date, close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = '2016'
),
daily_returns AS (
    SELECT
        symbol,
        trade_date,
        (close / LAG(close) OVER (PARTITION BY symbol ORDER BY trade_date) - 1.0) AS daily_return
    FROM daily_2016
),
stats AS (
    SELECT
        symbol,
        COUNT(daily_return) AS n_returns,
        AVG(daily_return) AS avg_daily_return,
        SQRT(AVG(daily_return * daily_return) - AVG(daily_return) * AVG(daily_return)) AS daily_vol
    FROM daily_returns
    WHERE daily_return IS NOT NULL
    GROUP BY symbol
    HAVING n_returns >= 200 AND daily_vol > 0
),
sharpe_like AS (
    SELECT
        symbol,
        (avg_daily_return / daily_vol) * SQRT(252) AS sharpe_like_2016,
        avg_daily_return,
        daily_vol
    FROM stats
)
SELECT
    sec."GICS sector" AS gics_sector,
    COUNT(*) AS n_stocks,
    ROUND(AVG(sh.sharpe_like_2016), 3) AS avg_sharpe_like_2016,
    ROUND(AVG(sh.avg_daily_return) * 252 * 100, 2) AS approx_annual_return_pct,
    ROUND(AVG(sh.daily_vol) * SQRT(252) * 100, 2) AS avg_annualized_vol_pct
FROM sharpe_like sh
INNER JOIN 'securities.csv' sec
    ON sec."Ticker symbol" = sh.symbol
GROUP BY gics_sector
HAVING n_stocks >= 5
ORDER BY avg_sharpe_like_2016 DESC;

,gics_sector,n_stocks,avg_sharpe_like_2016,approx_annual_return_pct,avg_annualized_vol_pct
0,Financials,62,0.910,22.77,25.99
1,Industrials,68,0.893,22.78,27.01
2,Energy,36,0.838,37.93,46.04
3,Utilities,28,0.822,16.01,20.18
4,Materials,25,0.760,24.36,30.67
5,Information Technology,68,0.729,19.74,28.53
6,Telecommunications Services,5,0.470,5.35,27.35
7,Consumer Staples,36,0.290,5.61,21.03
8,Consumer Discretionary,84,0.268,6.41,29.41
9,Real Estate,29,0.204,4.97,23.23


**_Insight :_**

- Even though Energy had high returns, it also had extremely high vol → still decent Sharpe-like, but not #1.
- Financials and Industrials lead on risk-adjusted terms (good returns without extreme volatility).

**6) Growth: Which companies had the strongest revenue CAGR (2012 → 2015)?**

_Question :_ Detect top fundamental growth stories (sales expansion).

In [7]:
-- Top 10 revenue CAGRs between 2012 and 2015 (3-year CAGR)
WITH latest_per_year AS (
    SELECT
        "Ticker symbol",
        CAST("For Year" AS INTEGER) AS fiscal_year,
        MAX("Period ending") AS period_ending
    FROM 'fundamentals.csv'
    WHERE "For Year" IN (2012, 2015)
    GROUP BY "Ticker symbol", CAST("For Year" AS INTEGER)
),
fund_clean AS (
    SELECT
        f."Ticker symbol" AS ticker_symbol,
        CAST(f."For Year" AS INTEGER) AS fiscal_year,
        f."Total revenue" AS total_revenue
    FROM 'fundamentals.csv' f
    INNER JOIN latest_per_year lp
        ON lp."Ticker symbol" = f."Ticker symbol"
       AND lp.fiscal_year = CAST(f."For Year" AS INTEGER)
       AND lp.period_ending = f."Period ending"
    WHERE total_revenue IS NOT NULL AND total_revenue > 0
),
rev_pivot AS (
    SELECT
        ticker_symbol,
        SUM(CASE WHEN fiscal_year = 2012 THEN total_revenue END) AS rev_2012,
        SUM(CASE WHEN fiscal_year = 2015 THEN total_revenue END) AS rev_2015
    FROM fund_clean
    GROUP BY ticker_symbol
    HAVING rev_2012 IS NOT NULL AND rev_2015 IS NOT NULL
),
cagr AS (
    SELECT
        r.ticker_symbol AS symbol,
        r.rev_2012,
        r.rev_2015,
        (pow(r.rev_2015 / r.rev_2012, 1.0 / 3) - 1.0) AS rev_cagr
    FROM rev_pivot r
)
SELECT
    c.symbol,
    s.security,
    s."GICS sector" AS gics_sector,
    ROUND(c.rev_2012 / 1e9, 2) AS rev_2012_usd_bn,
    ROUND(c.rev_2015 / 1e9, 2) AS rev_2015_usd_bn,
    ROUND(c.rev_cagr * 100, 2) AS rev_cagr_pct
FROM cagr c
LEFT JOIN 'securities.csv' s
    ON s."Ticker symbol" = c.symbol
ORDER BY rev_cagr_pct DESC
LIMIT 10;

,symbol,Security,gics_sector,rev_2012_usd_bn,rev_2015_usd_bn,rev_cagr_pct
0,GILD,Gilead Sciences,Health Care,9.70,32.64,49.84
1,CNC,Centene Corporation,Health Care,8.11,22.76,41.05
2,ESS,"Essex Property Trust, Inc.",Real Estate,0.54,1.19,30.68
3,MAA,Mid-America Apartments,Real Estate,0.48,1.04,29.89
4,UA,Under Armour,Consumer Discretionary,1.83,3.96,29.26
5,HCN,Welltower Inc.,Real Estate,1.77,3.78,28.83
6,O,Realty Income Corporation,Real Estate,0.48,1.02,28.30
7,DLTR,Dollar Tree,Consumer Discretionary,7.39,15.50,27.98
8,TRIP,TripAdvisor,Consumer Discretionary,0.76,1.49,25.05
9,EXR,Extra Space Storage,Real Estate,0.41,0.78,24.09


**_Insight :_**

- Growth leaders cluster in Health Care and Real Estate (plus a few consumer names).
- This is pure top-line growth; it does not guarantee profitability or stock outperformance.

**7) Profitability: Which sectors earned the highest net margin in 2015 (weighted)?**

_Question :_ Measure sector profitability using weighted net margin = total net income / total revenue.

In [8]:
-- Weighted net margin by sector in 2015 (using latest record per ticker for 2015)
WITH latest_2015 AS (
    SELECT "Ticker symbol" AS ticker_symbol, MAX("Period Ending") AS period_ending
    FROM 'fundamentals.csv'
    WHERE "For year" = 2015
    GROUP BY ticker_symbol
),
fund_2015 AS (
    SELECT
        f."Ticker symbol" AS symbol,
        f."Total revenue" AS total_revenue,
        f."Net income" AS net_income
    FROM 'fundamentals.csv' f
    INNER JOIN latest_2015 l
        ON l.ticker_symbol = f."Ticker symbol" AND l.period_ending = f."Period ending"
    WHERE f."Total revenue" IS NOT NULL AND f."Total revenue" > 0
      AND f."Net income" IS NOT NULL
)
SELECT
    s."GICS sector" AS gics_sector,
    COUNT(*) AS n_companies,
    ROUND(SUM(f.net_income) / 1e9, 2) AS net_income_usd_bn,
    ROUND(SUM(f.total_revenue) / 1e9, 2) AS revenue_usd_bn,
    ROUND((SUM(f.net_income) * 1.0 / SUM(f.total_revenue)) * 100, 2) AS net_margin_pct
FROM fund_2015 f
INNER JOIN 'securities.csv' s
    ON s."Ticker symbol" = f.symbol
GROUP BY gics_sector
HAVING n_companies >= 5
ORDER BY net_margin_pct DESC;

,gics_sector,n_companies,net_income_usd_bn,revenue_usd_bn,net_margin_pct
0,Real Estate,26,13.96,73.79,18.91
1,Information Technology,57,150.18,857.64,17.51
2,Financials,50,141.89,889.69,15.95
3,Telecommunications Services,5,35.34,310.13,11.40
4,Industrials,61,97.45,1004.51,9.70
5,Utilities,23,24.52,254.46,9.64
6,Health Care,47,84.51,1209.26,6.99
7,Consumer Discretionary,73,95.87,1430.27,6.70
8,Consumer Staples,30,73.64,1420.12,5.19
9,Materials,24,9.95,267.48,3.72


_**Insight :**_

- 2015 stands out for Energy with negative net margin, consistent with commodity downturn dynamics.
- Tech and Financials look structurally profitable on a weighted basis.

**8) Pattern: Do less-profitable firms rebound more in 2016? (2015 margin buckets vs 2016 returns)**

_Question :_ Test a market pattern: “distressed” (negative margin) firms may outperform the next year (mean reversion / rebound).

In [9]:
-- 2015 net margin buckets vs average 2016 total return
WITH prices_2016 AS (
    SELECT symbol, date AS trade_date, close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date) = '2016'
),
first_last AS (
    SELECT symbol, MIN(trade_date) AS first_date, MAX(trade_date) AS last_date
    FROM prices_2016
    GROUP BY symbol
),
ret2016 AS (
    SELECT
        fl.symbol,
        (p_last.close / p_first.close - 1.0) AS ret_2016
    FROM first_last fl
    INNER JOIN prices_2016 p_first
        ON p_first.symbol = fl.symbol AND p_first.trade_date = fl.first_date
    INNER JOIN prices_2016 p_last
        ON p_last.symbol = fl.symbol AND p_last.trade_date = fl.last_date
),
latest_2015 AS (
    SELECT "Ticker symbol" AS ticker_symbol, MAX("Period ending") AS period_ending
    FROM 'fundamentals.csv'
    WHERE "For year" = 2015
    GROUP BY "Ticker symbol"
),
margin_2015 AS (
    SELECT
        f."Ticker symbol" AS symbol,
        (f."Net income" * 1.0 / f."Total revenue") AS margin_2015
    FROM 'fundamentals.csv' f
    INNER JOIN latest_2015 l
        ON l.ticker_symbol = f."Ticker symbol" AND l.period_ending = f."Period ending"
    WHERE f."Total revenue" IS NOT NULL AND f."Total revenue" > 0
      AND f."Net income" IS NOT NULL
),
joined AS (
    SELECT
        r.symbol,
        r.ret_2016,
        m.margin_2015
    FROM ret2016 r
    INNER JOIN margin_2015 m
        ON m.symbol = r.symbol
)
SELECT
    CASE
        WHEN margin_2015 < 0 THEN 'Negative'
        WHEN margin_2015 < 0.05 THEN '0–5%'
        WHEN margin_2015 < 0.10 THEN '5–10%'
        WHEN margin_2015 < 0.20 THEN '10–20%'
        ELSE '20%+'
    END AS margin_bucket,
    COUNT(*) AS n_stocks,
    ROUND(AVG(margin_2015) * 100, 2) AS avg_margin_2015_pct,
    ROUND(AVG(ret_2016) * 100, 2) AS avg_ret_2016_pct,
    ROUND(MIN(ret_2016) * 100, 2) AS worst_ret_pct,
    ROUND(MAX(ret_2016) * 100, 2) AS best_ret_pct
FROM joined
GROUP BY margin_bucket
HAVING n_stocks >= 10
ORDER BY avg_margin_2015_pct;

,margin_bucket,n_stocks,avg_margin_2015_pct,avg_ret_2016_pct,worst_ret_pct,best_ret_pct
0,Negative,35,-51.40,26.85,-40.05,154.71
1,0–5%,68,3.27,14.07,-27.85,126.47
2,5–10%,111,7.69,11.41,-34.64,63.12
3,10–20%,147,14.16,13.96,-51.90,229.75
4,20%+,63,28.41,10.08,-29.37,48.42


**_Insight :_**

- The negative-margin bucket has the highest average 2016 return → consistent with a rebound/mean-reversion dynamic.
- Very high margins (20%+) did not correspond to highest next-year returns → markets may already price “quality”.

**9) Balance-sheet risk: Which sectors were most leveraged in 2015, and how volatile were they in 2016?**

_Question :_ Connect fundamentals risk (Debt/Assets) with market risk (volatility).

In [10]:
-- Sector leverage (2015 Debt/Assets) vs sector average volatility (2016)
WITH daily_2016 AS (
    SELECT symbol, date AS trade_date, close
    FROM 'prices-split-adjusted.csv'
    WHERE EXTRACT(YEAR FROM date)= '2016'
),
daily_returns AS (
    SELECT
        symbol,
        trade_date,
        (close / LAG(close) OVER (PARTITION BY symbol ORDER BY trade_date) - 1.0) AS daily_return
    FROM daily_2016
),
vol_2016 AS (
    SELECT
        symbol,
        COUNT(daily_return) AS n_returns,
        SQRT(AVG(daily_return * daily_return) - AVG(daily_return) * AVG(daily_return)) AS daily_vol
    FROM daily_returns
    WHERE daily_return IS NOT NULL
    GROUP BY symbol
    HAVING n_returns >= 200 AND daily_vol > 0
),
latest_2015 AS (
    SELECT "Ticker symbol" AS ticker_symbol, MAX("Period ending") AS period_ending
    FROM 'fundamentals.csv'
    WHERE "For year" = 2015
    GROUP BY "Ticker symbol"
),
leverage_2015 AS (
    SELECT
        f."Ticker symbol" AS symbol,
        (f."Total liabilities" * 1.0 / f."Total assets") AS debt_to_assets_2015
    FROM 'fundamentals.csv' f
    INNER JOIN latest_2015 l
        ON l.ticker_symbol = f."Ticker symbol" AND l.period_ending = f."Period ending"
    WHERE f."Total assets" IS NOT NULL AND f."Total assets" > 0
      AND f."Total liabilities" IS NOT NULL AND f."Total liabilities" >= 0
),
joined AS (
    SELECT
        v.symbol,
        v.daily_vol * SQRT(252) AS vol_2016,
        lev.debt_to_assets_2015
    FROM vol_2016 v
    INNER JOIN leverage_2015 lev
        ON lev.symbol = v.symbol
)
SELECT
    s."GICS sector" AS gics_sector,
    COUNT(*) AS n_stocks,
    ROUND(AVG(j.debt_to_assets_2015) * 100, 2) AS debt_to_assets_2015_pct,
    ROUND(AVG(j.vol_2016) * 100, 2) AS vol_2016_pct
FROM joined j
INNER JOIN 'securities.csv' s
    ON s."Ticker symbol" = j.symbol
GROUP BY s."GICS sector"
HAVING n_stocks >= 5
ORDER BY debt_to_assets_2015_pct DESC;

,gics_sector,n_stocks,debt_to_assets_2015_pct,vol_2016_pct
0,Financials,50,83.72,25.64
1,Telecommunications Services,5,74.12,27.35
2,Utilities,23,72.74,18.66
3,Consumer Staples,30,72.25,20.64
4,Materials,24,67.60,31.12
5,Industrials,61,66.65,27.21
6,Consumer Discretionary,72,64.32,29.26
7,Real Estate,26,62.38,23.32
8,Health Care,47,62.34,26.55
9,Energy,29,58.68,46.12


_**Insight :**_

- Financials are most leveraged by design (balance-sheet businesses) but not the most volatile here.
- Energy shows extremely high volatility despite lower average leverage. It is because for this sector market risk is driven more by commodity exposure than leverage alone.

**Conclusion :**

Across 2016, Energy and Materials led on raw returns, while Financials and Industrials looked strongest on risk-adjusted performance. Liquidity was concentrated in a small set of mega-caps (Tech + big banks), which matters for execution and portfolio capacity. On fundamentals, Energy had negative sector net margins in 2015, and the 2015→2016 analysis suggests a rebound effect: negative-margin firms delivered the highest average returns the next year. Leverage varies strongly by sector (Financials highest), but volatility is not purely explained by leverage (Energy is the clearest counterexample).

**Limitations :**

- Returns exclude dividends (not total shareholder return).
- Fundamentals coverage is much richer for 2012–2015 than 2016 in this dataset.
- Accounting metrics can be noisy/outlier-prone (impairments, one-offs), so interpretation benefits from winsorization or robust stats (not shown here).